In [23]:
pip install python-dotenv pytz

Note: you may need to restart the kernel to use updated packages.


In [17]:
#Importing required libraries

import http.client
import json
import csv
import time
import os
from datetime import datetime
from zoneinfo import ZoneInfo   
from dotenv import load_dotenv

In [23]:
#Loading API key and TikTok user ID from .env file

load_dotenv()

API_KEY  = os.getenv("SCRAPTIK_API_KEY")
USER_ID  = os.getenv("TIKTOK_USER_ID")
CSV_PATH = os.path.expanduser(r"C:\Users\YUSAB\ABIDEMI\datawithdemi_tiktok.csv")

TZ       = ZoneInfo("Africa/Lagos")  # Nigeria time

#Stage‑check
print(f"API key loaded? {'✅' if API_KEY else '❌'}")
print(f"User ID       : {'✅' or '❌ MISSING'}")
print(f"CSV save path : {CSV_PATH}")

API key loaded? ✅
User ID       : ✅
CSV save path : C:\Users\YUSAB/Downloads/datawithdemi_tiktok.csv


In [25]:
#log errors to a text file (optional but useful for debugging)

def log_error(msg: str) -> None:
    """Append errors to a simple text log."""
    with open("task_error_log.txt", "a", encoding="utf-8") as f:
        f.write(f"[{datetime.now()}] {msg}\n")
        
#Check if previous posts already exist in the CSV file

def load_existing_posts() -> dict:
    """Return dict keyed by post‑ID so we can skip duplicates."""
    posts = {}
    if os.path.exists(CSV_PATH):
        with open(CSV_PATH, newline="", encoding="utf-8") as f:
            rdr = csv.reader(f)
            _ = next(rdr, None)          # skip header
            for row in rdr:
                posts[row[0]] = row
    return posts
#Save all the scraped posts into a CSV file
def save_posts_to_csv(posts: dict) -> None:
    """Write entire dict back to CSV (overwrites)."""
    with open(CSV_PATH, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow([
            "Post ID", "Description", "Likes", "Comments", "Shares", "Views",
            "Video URL", "Date (Lagos)", "Duration (s)", "Duration Category",
            "Engagement Rate", "Sound Title", "Region", "Hashtags"
        ])
        for row in posts.values():
            w.writerow(row)


In [36]:
#Fetch and process posts using Scraptik API

def fetch_user_posts(api_key: str, user_id: str) -> None:
    conn    = http.client.HTTPSConnection("scraptik.p.rapidapi.com")
    headers = {"x-rapidapi-key": api_key, "x-rapidapi-host": "scraptik.p.rapidapi.com"}

    max_cursor  = 0
    has_more    = True
    posts_dict  = load_existing_posts()
    stage_rows0 = len(posts_dict)

    print(f"🔎 Existing posts in CSV: {stage_rows0}")

    while has_more:
        conn.request(
            "GET",
            f"/user-posts?user_id={user_id}&count=50&max_cursor={max_cursor}",
            headers=headers,
        )
        res   = conn.getresponse()
        code  = res.status
        blob  = res.read()
        if code != 200:
            msg = f"API error {code}"
            print("❌", msg)
            log_error(msg)
            break

        payload = json.loads(blob)
        if "aweme_list" not in payload:
            print("❌ Unexpected response structure.")
            break

        for aweme in payload["aweme_list"]:
            pid = aweme.get("aweme_id")
            if not pid or pid in posts_dict:
                continue  # skip duplicates or missing IDs

            stats   = aweme.get("statistics", {})
            video   = aweme.get("video",  {})
            music   = aweme.get("music",  {})
            
            # Classify video duration
            duration_s   = video.get("duration", 0) / 1000
            duration_cat = "Short" if duration_s < 15 else "Medium" if duration_s <= 60 else "Long"
            eng_rate     = (
                (stats.get("digg_count",0)+stats.get("comment_count",0)+stats.get("share_count",0))
                / max(stats.get("play_count",1), 1)
            )
            hashtags = ", ".join(
                tag["hashtag_name"]
                for tag in aweme.get("text_extra", [])
                if tag.get("hashtag_name")
            )

            # date conversion to Africa/Lagos
            timestamp  = aweme.get("create_time", 0)
            lagos_time = datetime.fromtimestamp(timestamp, TZ).strftime("%Y-%m-%d %H:%M:%S")

            posts_dict[pid] = [
                pid,
                aweme.get("desc",""),
                stats.get("digg_count",0),
                stats.get("comment_count",0),
                stats.get("share_count",0),
                stats.get("play_count",0),
                video.get("play_addr",{}).get("url_list",[ ""])[0],
                lagos_time,
                round(duration_s,2),
                duration_cat,
                round(eng_rate,4),
                music.get("title","N/A"),
                aweme.get("region","N/A"),
                hashtags
            ]

        has_more   = payload.get("has_more", False)
        max_cursor = payload.get("max_cursor", 0)
        time.sleep(1)  # gentle throttle

    #Save
    save_posts_to_csv(posts_dict)
    added = len(posts_dict) - stage_rows0
    print(f"✅ Scrape complete — added {added} new post(s). Total now: {len(posts_dict)}")


In [38]:
#Start the scraping process

if API_KEY and USER_ID:
    print("🚀 Starting TikTok scraping…")
    fetch_user_posts(API_KEY, USER_ID)
else:
    print("❌ Fix .env — missing API_KEY or USER_ID")


🚀 Starting TikTok scraping…
🔎 Existing posts in CSV: 0
❌ API error 429
✅ Scrape complete — added 0 new post(s). Total now: 0


In [34]:

import pandas as pd

df = pd.read_csv(CSV_PATH)
print(f"Rows in file  : {len(df)}")
print("Most recent 5 rows:")
display(df.tail())


Rows in file  : 0
Most recent 5 rows:


,Post ID,Description,Likes,Comments,Shares,Views,Video URL,Date (Lagos),Duration (s),Duration Category,Engagement Rate,Sound Title,Region,Hashtags
